In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
# مسیرها
csv_path = ".\\bee\\bee_data.csv"          # مسیر فایل CSV شما
images_folder = ".\\bee\\bee_imgs"         # مسیر پوشه تصاویر

# خواندن CSV
df = pd.read_csv(csv_path)

# آماده کردن لیست ها
X = []
y = []

# پردازش هر رکورد
for idx, row in df.iterrows():
    img_name = row['file']    # نام ستون فایل عکس در CSV
    label = row['pollen_carrying']         # نام ستون تارگت در CSV

    img_path = Path(images_folder) / img_name
    img = cv2.imread(str(img_path))
    
    if img is None:
        print(f"تصویر {img_path} پیدا نشد!")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # تبدیل BGR به RGB
    img = cv2.resize(img, (128, 128))          # تغییر اندازه تصاویر به 128x128 (اختیاری)
    img = img / 255.0                          # نرمال‌سازی به [0,1]

    X.append(img)
    y.append(label)

# تبدیل به numpy array
X = np.array(X, dtype=np.float32)
y = np.array(y)

# تقسیم داده به آموزش و آزمون
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("اندازه داده‌ها:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

اندازه داده‌ها:
X_train: (4137, 128, 128, 3)
X_test: (1035, 128, 128, 3)
y_train: (4137,)
y_test: (1035,)


In [ ]:
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Add

output_activation = 'sigmoid'
loss_fn = 'binary_crossentropy'

# تعریف یک Residual Block
def residual_block(x, filters):
    shortcut = x
    x = Conv2D(filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)

    # skip connection
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

# ورودی
inputs = Input(shape=(128,128,3))

# Block 1
x = Conv2D(16, (3,3), padding='same', activation='relu')(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# Residual Block 1
x = residual_block(x, 16)

# Block 2
x = Conv2D(32, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# Residual Block 2
x = residual_block(x, 32)

# Block 3
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)

# Residual Block 3
x = residual_block(x, 64)

# Global pooling + Dense
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation=output_activation)(x)

# ساخت مدل
model = Model(inputs, outputs)

# کامپایل
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

# خلاصه مدل
model.summary()

# آموزش
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=32
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │        448 │ input_layer_2[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │         64 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 64, 64,    │      2,320 │ max_pooling2d_3[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │         64 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 64, 64,    │      2,320 │ activation_4[0][… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │         64 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 64, 64,    │          0 │ add_3[0][0]       │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 64, 64,    │      4,640 │ activation_5[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        128 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │      9,248 │ max_pooling2d_4[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        128 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 32, 32,    │          0 │ batch_normalizat

 Total params: 130,369 (509.25 KB)

 Trainable params: 129,697 (506.63 KB)

 Non-trainable params: 672 (2.62 KB)

Epoch 1/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 30s 202ms/step - accuracy: 0.9911 - loss: 0.0378 - val_accuracy: 0.9961 - val_loss: 0.0587
Epoch 2/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 29s 223ms/step - accuracy: 0.9966 - loss: 0.0214 - val_accuracy: 0.9961 - val_loss: 0.0318
Epoch 3/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 30s 228ms/step - accuracy: 0.9966 - loss: 0.0230 - val_accuracy: 0.9961 - val_loss: 0.0348
Epoch 4/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 29s 222ms/step - accuracy: 0.9966 - loss: 0.0209 - val_accuracy: 0.9961 - val_loss: 0.0346
Epoch 5/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 29s 221ms/step - accuracy: 0.9966 - loss: 0.0238 - val_accuracy: 0.9961 - val_loss: 0.0350
Epoch 6/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 29s 222ms/step - accuracy: 0.9966 - loss: 0.0228 - val_accuracy: 0.9961 - val_loss: 0.0275
Epoch 7/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 29s 222ms/step - accuracy: 0.9966 - loss: 0.0228 - val_accuracy: 0.9961 - val_loss: 0.0376
Epoch 8/15
130/130 ━━━━━━━━━━━━━━━━━━━━ 29s 224ms/step - accuracy: 0.9966 - loss: 0

In [6]:
model.save('bee_ResNet.keras')